In [1]:
import networkx as nx
from gerrychain import Graph
import math
import time
import gurobipy as gp
from gurobipy import GRB

from util import update_attributes, get_k_L_U
from cluster import is_within_window, find_cluster
from separation import labeling_contiguity_callback
filepath = 'C:\\districting-data-2020\\'

In [2]:
# finds a nontrivial county clustering (S,V\S) of the vertices 
#   (if one exists) under 1-person population deviation
#
def find_one_person_deviation_clustering(G, k):
    
    # set L and U
    total_population = sum( G.nodes[i]['TOTPOP'] for i in G.nodes )
    L = math.floor( total_population / k )
    U = math.ceil( total_population / k )
    
    # first, check if some county plus a subset of its neighborhood fits the bill
    for v in G.nodes:
        neighborhood = list( G.neighbors(v) ) + [v]
        (cluster, size) = find_cluster(G.subgraph( neighborhood ), [v], L, U, k, verify=False)
        if cluster is not None:
            complement = [ i for i in G.nodes if i not in cluster ]
            if len(complement) > 0 and nx.is_connected( G.subgraph(complement) ):
                print("Found a neighborhood solution around",G.nodes[v]['NAME20'],"County.")
                return ( [cluster, complement], [size, k-size] )
            
    # second, check if some edge plus a subset of its neighborhood fits the bill
    for u,v in G.edges:
        neighborhood = set( list(G.neighbors(u)) + list(G.neighbors(v)) )                       
        (cluster, size) = find_cluster(G.subgraph( neighborhood ), [u,v], L, U, k, verify=False)
        if cluster is not None:
            complement = [ i for i in G.nodes if i not in cluster ]
            if len(complement) > 0 and nx.is_connected( G.subgraph(complement) ):
                print("Found a neighborhood solution around",G.nodes[u]['NAME20'],"and",G.nodes[v]['NAME20'],"Counties.")
                return ( [cluster, complement], [size, k-size] )

    # build model
    m = gp.Model()
    
    # x[i,j]=1 if vertex i is assigned to cluster j
    x = m.addVars(G.nodes, 2, vtype=GRB.BINARY)
    m.addConstrs( x[i,0] + x[i,1] == 1 for i in G.nodes )
    
    # y[j] is size of cluster j
    y = m.addVars(2, vtype=GRB.INTEGER)
    m.addConstr( y[0] + y[1] == k )
    y[1].LB = 1
    
    # add population balance constraints
    for j in range(2):
        m.addConstr( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) >= L * y[j]  )
        m.addConstr( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) <= U * y[j]  )
        
    # symmetry breaking: fix a particular vertex to be in S, say one with extreme population.
    mpop = min( G.nodes[i]['TOTPOP'] for i in G.nodes )
    mpv = [ i for i in G.nodes if G.nodes[i]['TOTPOP']==mpop ][0]
    x[mpv,0].LB = 1
    
    # add flow-based contiguity constraints for S (which is rooted at mpv)
    M = G.number_of_nodes() - 1
    DG = nx.DiGraph(G)
    f = m.addVars(DG.edges)
    m.addConstrs( gp.quicksum( f[j,i] - f[i,j] for j in G.neighbors(i) ) == x[i,0] for i in G.nodes if i != mpv )
    m.addConstrs( gp.quicksum( f[j,i] for j in G.neighbors(i) ) <= M * x[i,0] for i in G.nodes if i != mpv )
    m.addConstr( gp.quicksum( f[j,mpv] for j in G.neighbors(mpv) ) == 0 )
    
    # add cut-based contiguity constraints for V\S (whose root we do not know a priori)
    m.Params.LazyConstraints = 1
    m._x = x
    m._DG = DG
    m._parts = 2
    
    # add compactness objective: a transportation objective for cluster S, with root at mpv.
    dist = nx.shortest_path_length(G, source=mpv)
    m.setObjective( gp.quicksum( dist[i] * dist[i] * round(G.nodes[i]['TOTPOP']/1000) * x[i,0] for i in G.nodes ), GRB.MINIMIZE )
    
    # solve
    m.Params.IntFeasTol = 1e-7
    m.Params.FeasibilityTol = 1e-7
    m.Params.MIPGap = 1.00  # use a large MIP gap, as we are only interested in feasibility
    
    # try each possible size for cluster S
    for size in range(1,k):
        y[0].LB = size
        y[0].UB = size
        m._callback = labeling_contiguity_callback 
        m.optimize( m._callback )
        
        if m.solCount > 0:
            cluster = [ i for i in G.nodes if x[i,0].x > 0.5 ]
            size = round( y[0].x )
            complement = [ i for i in G.nodes if x[i,1].x > 0.5 ]
            return ( [cluster, complement], [ size, k-size ] )
    
    return ( [list(G.nodes)], [k] )

In [3]:
from number_of_districts import congressional_districts_2020
states = sorted([ state for state in congressional_districts_2020.keys() ])

results = dict()

for district_type in ['CD', 'SS', 'SH']:
    
    print("**********************************")
    print("**********************************")
    print("District_type:",district_type)
    print("**********************************")
    print("**********************************")
    print("")
    
    for state in states:   
        
        print("**********************************")
        print("State:",state)
        print("**********************************")

        filename = state + '_county.json'
        GC = Graph.from_json( filepath + filename )
        update_attributes(GC, state)    

        (k, L, U) = get_k_L_U(GC, state, district_type)
        if k <= 1 or not nx.is_connected(GC):
            print("Skipping this state because k <= 1 or because G is disconnected.")
            continue

        (clusters, sizes) = find_one_person_deviation_clustering(GC, k)
        results[state, district_type] = dict()
        results[state, district_type]['clusters'] = clusters
        results[state, district_type]['sizes'] = sizes


**********************************
**********************************
District_type: CD
**********************************
**********************************

**********************************
State: AK
**********************************
Starting AK with k = 1 and deviation = 0.01
Thus, we have L = 729725 and U = 737057
Skipping this state because k <= 1 or because G is disconnected.
**********************************
State: AL
**********************************
Starting AL with k = 7 and deviation = 0.01
Thus, we have L = 714166 and U = 721342
Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-27
Set parameter LazyConstraints to value 1
Set parameter IntFeasTol to value 1e-07
Set parameter FeasibilityTol to value 1e-07
Set parameter MIPGap to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 205 rows, 478 columns and 1556 nonzeros
Model f


User-callback calls 174646, time in user-callback 0.41 sec
**********************************
State: AZ
**********************************
Starting AZ with k = 9 and deviation = 0.01
Thus, we have L = 790639 and U = 798584
Set parameter LazyConstraints to value 1
Set parameter IntFeasTol to value 1e-07
Set parameter FeasibilityTol to value 1e-07
Set parameter MIPGap to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 49 rows, 96 columns and 310 nonzeros
Model fingerprint: 0xbffa8362
Variable types: 64 continuous, 32 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+06]
  Objective range  [4e+01, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 25 rows and 47 columns
Presolve time: 0.00s
Presolved: 24 rows, 49 columns, 154 nonzeros
Variable types: 37 continuous, 12 integer (12 binary)

Root relaxation: objecti

     0     0 7222.27152    0   10          - 7222.27152      -     -    0s
     0     0 7228.57240    0   10          - 7228.57240      -     -    0s
     0     0 7235.82914    0    9          - 7235.82914      -     -    0s
     0     0 7244.57335    0   11          - 7244.57335      -     -    0s
     0     0 7244.57335    0   11          - 7244.57335      -     -    0s
     0     0 7244.57335    0   11          - 7244.57335      -     -    0s
     0     2 7244.57335    0   11          - 7244.57335      -     -    0s

Cutting planes:
  MIR: 7
  StrongCG: 3
  Flow cover: 3

Explored 42 nodes (494 simplex iterations) in 0.17 seconds (0.02 work units)
Thread count was 8 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

User-callback calls 598, time in user-callback 0.01 sec
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 49 rows

     0     0 45916.2723    0    7          - 45916.2723      -     -    0s
     0     0 45916.2723    0    7          - 45916.2723      -     -    0s
     0     0 45927.6860    0    8          - 45927.6860      -     -    0s
     0     0 45931.8869    0    9          - 45931.8869      -     -    0s
     0     0 45931.8869    0    9          - 45931.8869      -     -    0s
     0     0 46010.9600    0    5          - 46010.9600      -     -    0s
     0     0 46012.8563    0    7          - 46012.8563      -     -    0s
     0     0 46016.2596    0    9          - 46016.2596      -     -    0s
     0     0 46016.2596    0    9          - 46016.2596      -     -    0s
     0     0 46036.6301    0    8          - 46036.6301      -     -    0s
     0     0 46039.9259    0    9          - 46039.9259      -     -    0s
     0     0 46165.1491    0    8          - 46165.1491      -     -    0s
     0     0 46165.1491    0    8          - 46165.1491      -     -    0s
     0     0 46183.2063  


Cutting planes:
  Cover: 1
  MIR: 6
  StrongCG: 3
  Flow cover: 6
  Network: 1
  Relax-and-lift: 1

Explored 62 nodes (549 simplex iterations) in 0.22 seconds (0.02 work units)
Thread count was 8 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -

User-callback calls 593, time in user-callback 0.01 sec
**********************************
State: CA
**********************************
Starting CA with k = 52 and deviation = 0.01
Thus, we have L = 756549 and U = 764152
Set parameter LazyConstraints to value 1
Set parameter IntFeasTol to value 1e-07
Set parameter FeasibilityTol to value 1e-07
Set parameter MIPGap to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 178 rows, 402 columns and 1310 nonzeros
Model fingerprint: 0xd8a6fa48
Variable types: 284 continuous, 118 integer (116 binary)
Coefficient statistics:
  Matrix range 

...

Presolve removed 32 rows and 70 columns
Presolve time: 0.01s
Presolved: 41 rows, 80 columns, 291 nonzeros
Variable types: 61 continuous, 19 integer (19 binary)

Root relaxation: objective 9.380003e+01, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   93.80003    0    1          -   93.80003      -     -    0s
     0     0   95.67250    0    4          -   95.67250      -     -    0s
     0     0  106.84375    0    4          -  106.84375      -     -    0s
     0     0  128.04193    0    6          -  128.04193      -     -    0s
     0     0  137.76255    0    4          -  137.76255      -     -    0s
     0     0  138.11910    0    5          -  138.11910      -     -    0s
     0     0  138.19047    0    6          -  138.19047      -     -    0s
     0     0  150.46846    0    7          -  150.46846      -     -    0s
 

     0     0  190.23789    0   11          -  190.23789      -     -    0s
     0     0  194.07037    0    8          -  194.07037      -     -    0s
     0     0  196.87025    0    9          -  196.87025      -     -    0s
     0     0  197.81608    0    9          -  197.81608      -     -    0s
     0     0  198.06496    0   10          -  198.06496      -     -    0s
     0     0  198.36098    0   10          -  198.36098      -     -    0s
     0     0  199.75088    0    9          -  199.75088      -     -    0s
     0     0  200.24087    0   10          -  200.24087      -     -    0s
     0     0  200.24199    0   10          -  200.24199      -     -    0s
     0     0  200.77274    0   10          -  200.77274      -     -    0s
     0     0  201.21765    0   11          -  201.21765      -     -    0s
     0     0  201.45032    0   11          -  201.45032      -     -    0s
     0     0  202.82130    0   11          -  202.82130      -     -    0s
     0     0  204.37195  

     0     0  244.14941    0    7          -  244.14941      -     -    0s
     0     0  244.19088    0    7          -  244.19088      -     -    0s
     0     0  244.25882    0    7          -  244.25882      -     -    0s
     0     0  244.30270    0    7          -  244.30270      -     -    0s
     0     0  248.55324    0    6          -  248.55324      -     -    0s
     0     0  248.96967    0    2          -  248.96967      -     -    0s
     0     0  249.02953    0    2          -  249.02953      -     -    0s
     0     0  253.77810    0    7          -  253.77810      -     -    0s
     0     0  255.05201    0    7          -  255.05201      -     -    0s
     0     0  258.83497    0    9          -  258.83497      -     -    0s
     0     0  269.77646    0    5          -  269.77646      -     -    0s
     0     0  271.95382    0    6          -  271.95382      -     -    0s
     0     0  272.78925    0    6          -  272.78925      -     -    0s
     0     0  272.82124  

Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [7e+00, 5e+02]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 6e+01]
Presolve removed 29 rows and 39 columns
Presolve time: 0.01s
Presolved: 44 rows, 111 columns, 349 nonzeros
Variable types: 90 continuous, 21 integer (21 binary)

Root relaxation: objective 2.855843e+02, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  285.58432    0    1          -  285.58432      -     -    0s
     0     0  286.71412    0    4          -  286.71412      -     -    0s
     0     0  288.29622    0    4          -  288.29622      -     -    0s
     0     0  290.25237    0    4          -  290.25237      -     -    0s
     0     0  296.16176    0    7          -  296.16176      -     -    0s
     0     0  296.72534    0    7          -  296.72534      -   

     0     0  369.84806    0   10          -  369.84806      -     -    0s
     0     0  369.85836    0    9          -  369.85836      -     -    0s
     0     0  370.41787    0    9          -  370.41787      -     -    0s
     0     0  371.14313    0   10          -  371.14313      -     -    0s
     0     0  371.20441    0   10          -  371.20441      -     -    0s
     0     0  371.67411    0   10          -  371.67411      -     -    0s
     0     0  371.75492    0   11          -  371.75492      -     -    0s
     0     0  371.75628    0   11          -  371.75628      -     -    0s
     0     0  372.16117    0    9          -  372.16117      -     -    0s
     0     0  373.25646    0    9          -  373.25646      -     -    0s
     0     0  373.40097    0    9          -  373.40097      -     -    0s
     0     0  373.42105    0   10          -  373.42105      -     -    0s
     0     0  373.78979    0   10          -  373.78979      -     -    0s
     0     0  373.89346  

In [4]:
print("state type cluster?")
for (state, district_type) in results.keys():
    sizes = results[state,district_type]['sizes']
    print(state, district_type, len(sizes)>1 )

state type cluster?
AL CD True
AR CD True
AZ CD False
CA CD True
CO CD True
CT CD False
FL CD True
GA CD True
IA CD True
ID CD True
IL CD True
IN CD True
KS CD True
KY CD True
LA CD True
MA CD False
MD CD False
ME CD False
MI CD True
MN CD True
MO CD True
MS CD True
MT CD True
NC CD True
NE CD True
NH CD False
NJ CD False
NM CD True
NV CD False
NY CD True
OH CD True
OK CD True
OR CD True
PA CD True
RI CD False
SC CD True
TN CD True
TX CD True
UT CD False
VA CD True
WA CD True
WI CD True
WV CD True
AK SS False
AL SS True
AR SS True
AZ SS False
CA SS True
CO SS True
CT SS False
DE SS False
FL SS True
GA SS True
IA SS True
ID SS True
IL SS True
IN SS True
KS SS True
KY SS True
LA SS True
MA SS False
MD SS False
ME SS False
MI SS True
MN SS True
MO SS True
MS SS True
MT SS True
NC SS True
ND SS True
NE SS True
NH SS False
NJ SS False
NM SS True
NV SS False
NY SS True
OH SS True
OK SS True
OR SS True
PA SS True
RI SS False
SC SS True
SD SS True
TN SS True
TX SS True
UT SS True
VA SS True
VT

In [5]:
print("results =",results)

results = {('AL', 'CD'): {'clusters': [[1, 2, 9, 20, 23, 26, 33, 36, 37, 39, 40, 45, 46, 50, 56, 62, 65, 66], [0, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 24, 25, 27, 28, 29, 30, 31, 32, 34, 35, 38, 41, 42, 43, 44, 47, 48, 49, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 63, 64]], 'sizes': [1, 6]}, ('AR', 'CD'): {'clusters': [[8, 11, 19, 20, 21, 25, 27, 29, 32, 33, 34, 36, 40, 49, 56, 57, 64, 65, 66, 72], [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 22, 23, 24, 26, 28, 30, 31, 35, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 58, 59, 60, 61, 62, 63, 67, 68, 69, 70, 71, 73, 74]], 'sizes': [1, 3]}, ('AZ', 'CD'): {'clusters': [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]], 'sizes': [9]}, ('CA', 'CD'): {'clusters': [[0, 1, 9, 11, 14, 16, 24, 30, 33, 39, 47, 49, 50, 51, 55, 56], [2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 15, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 34, 35, 36, 37, 38, 40, 41, 42, 43, 44, 45, 46, 48, 52, 53

In [6]:
count_CD = sum( 1 for (state, district_type) in results.keys() if district_type=='CD' )
count_CD_true = sum( 1 for (state, district_type) in results.keys() if district_type=='CD' and len(results[state,district_type]['sizes'])>1 )
print("CD: ",count_CD_true,'/', count_CD)

count_SS = sum( 1 for (state, district_type) in results.keys() if district_type=='SS' )
count_SS_true = sum( 1 for (state, district_type) in results.keys() if district_type=='SS' and  len(results[state,district_type]['sizes'])>1 )
print("SS: ",count_SS_true,'/', count_SS)

count_SH = sum( 1 for (state, district_type) in results.keys() if district_type=='SH' )
count_SH_true = sum( 1 for (state, district_type) in results.keys() if district_type=='SH' and  len(results[state,district_type]['sizes'])>1 )
print("SH: ",count_SH_true,'/', count_SH)

CD:  33 / 43
SS:  37 / 49
SH:  40 / 48


In [9]:
# make table
for state in states:
    if state == 'HI':
        continue
    
    # state
    print(state, end='')
    
    # num counties
    clusters = results[state, 'SS']['clusters']
    num_counties = sum( len(cluster) for cluster in clusters )
    print(" &",num_counties, end='')
    
    for district_type in ['CD','SS','SH']:
        
        if (state, district_type) in results.keys():
            status = '\\greencheck' if len(results[state,district_type]['sizes'])>1 else '\\redx'
        else:
            status = '$\\blacksquare$'
            
        print(" &", status, end='')
        
    print("\\\\")

AK & 30 & $\blacksquare$ & \redx & \redx\\
AL & 67 & \greencheck & \greencheck & \greencheck\\
AR & 75 & \greencheck & \greencheck & \greencheck\\
AZ & 15 & \redx & \redx & \redx\\
CA & 58 & \greencheck & \greencheck & \greencheck\\
CO & 64 & \greencheck & \greencheck & \greencheck\\
CT & 8 & \redx & \redx & \greencheck\\
DE & 3 & $\blacksquare$ & \redx & \redx\\
FL & 67 & \greencheck & \greencheck & \greencheck\\
GA & 159 & \greencheck & \greencheck & \greencheck\\
IA & 99 & \greencheck & \greencheck & \greencheck\\
ID & 44 & \greencheck & \greencheck & \greencheck\\
IL & 102 & \greencheck & \greencheck & \greencheck\\
IN & 92 & \greencheck & \greencheck & \greencheck\\
KS & 105 & \greencheck & \greencheck & \greencheck\\
KY & 120 & \greencheck & \greencheck & \greencheck\\
LA & 64 & \greencheck & \greencheck & \greencheck\\
MA & 14 & \redx & \redx & \redx\\
MD & 24 & \redx & \redx & \redx\\
ME & 16 & \redx & \redx & \greencheck\\
MI & 83 & \greencheck & \greencheck & \greencheck\\
MN